# 1D Maze with Q Learning

In this notebook we attempt to solve the 1D Maze reinforcement learning problem. However, instead of using _TD Learning_ we will be using _Q Learning_. The difference is in the value function and the delta function. When using Q learning, one considers the value of a __state action pair__. Being in a state and going left has a different value than being in a state and going right. The Q function and delta function look like so: <br>
$\boldsymbol{Q}(\boldsymbol{s},\boldsymbol{a}) = \boldsymbol{r}(\boldsymbol{s} + 1) + \gamma \boldsymbol{r}(\boldsymbol{s} + 2) \ldots \gamma^{\boldsymbol{n}-1} \boldsymbol{r}(\boldsymbol{s}+\boldsymbol{n})$ <br>
$\delta(\boldsymbol{s},\boldsymbol{a}) = (\boldsymbol{r}(\boldsymbol{s}+1) + \gamma \boldsymbol{Q}(\boldsymbol{s}+1, \boldsymbol{a}+1)) - \boldsymbol{Q}(\boldsymbol{s}, \boldsymbol{a})$ <br>

The Q Table for a maze of length four would look like the following:

| State: | 1 | 2 | 3 | 4 |
| --- | --- | --- | --- | --- |
| Left | 1/2 | 1 | 1/2 | 1/4 |
| Right | 1/2 | 1/4 | 1/2 | 1 |

__Warning: This code is not yet functional__

In [121]:
import numpy as np
from sympy import *
init_printing(use_latex=True)
from fractions import Fraction

EPOCH = 20
GAMMA = .5
class Maze:
    def __init__(self, goal, length):
        self.reward = np.zeros(length)
        self.value = np.zeros((2,length))
        self.goal = goal
        self.length = length
        
        self.reward[goal] = 1
        
    # Performs one episode of learning on the maze
    # S is the starting state
    def episode(self, s):
        a = 0   # Must select an action to start with
        while s != self.goal:
            self.value[a, s] += self.delta(s,a)
            s, a = self.nextS(s, a)
    
    # delta(s,a) = r(s+1) + GAMMA * maxQ(s+1,a+1) - Q(s,a)
    def delta(self, s, a):
        # Obtain what the next state will be and the value
        
        return self.Q(s, a) - self.value[a,s]
    
    # Q(s,a) = r(s+1) + GAMMA * Q(s+1, a+1)
    def Q(self, s, a):
        nextS, nextA = self.nextS(s, a)
        future_value = self.reward[nextS] + GAMMA * self.value[nextA,nextS]
        return future_value
        
    # Determine which state is next
    def nextS(self, s, a):
        # Obtain which state is to the left and which is to the right
        if (s == 0):
            left = self.length-1
            right = s+1
        elif (s == self.length-1):
            left = s-1
            right = 0
        else:
            left = s-1
            right = s+1
        # Execute the action
        if (a == 0):
            nextS = left
        else:
            nextS = right
        
        # Which action will have the highest value?
        if (self.value[0, nextS] >= self.value[1, nextS]):
            nextA = 0
        else:
            nextA = 1
        return nextS, nextA
    
    # Print the maze 
    def display(self):
        print("The value matrix is:")
        print("L: ", end='')
        for i in self.value[0]:
            print( "|", Fraction(i), end='')
        print("|")
        print("R: ", end='')
        for i in self.value[1]:
            print( "|", Fraction(i), end='')
        print("|")
    

In [122]:
maze = Maze(0, 6)
maze.display()
for i in range(EPOCH):
    s = np.random.randint(0, maze.length)
    maze.episode(s)
    
maze.display()

The value matrix is:
L: | 0| 0| 0| 0| 0| 0|
R: | 0| 0| 0| 0| 0| 0|
The value matrix is:
L: | 0| 1| 1/2| 1/4| 1/8| 1/16|
R: | 0| 0| 0| 0| 0| 0|


In [123]:
maze.delta(3, 1)